<a href="https://colab.research.google.com/github/metekavakli1103/nmt_en_tr/blob/master/Cevirici.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kütüphaneleri dahil ettik

In [33]:
!pip install -q -U tensor2tensor
!pip install -q tensorflow matplotlib

In [15]:
import sys
if 'google.colab' in sys.modules: # Colab-only TensorFlow version selector
  %tensorflow_version 1.x
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.utils import trainer_lib
from tensor2tensor.data_generators import text_encoder

from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry

import textwrap


# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys



## Kodlarımı githubdan dahil ettik

In [16]:
!test -d nmt-en-tr || git clone https://github.com/metekavakli1103/nmt_en_tr.git

fatal: destination path 'nmt_en_tr' already exists and is not an empty directory.


In [17]:
import sys
sys.path.append("nmt_en_tr")
import nmt_en_tr

# **Ağırlıkları dahil edelim**

In [18]:
!wget -nc -q https://github.com/metekavakli1103/transtlate-tr-eng/raw/mete/en2tr.zip
!unzip -n -qq en2tr.zip

In [19]:
model_path = 'en2tr'

data_dir = os.path.join(model_path, 'data')
ckpt_dir = os.path.join(model_path, 'model')

en2tr_problem = problems.problem("translate_en_tr")
encoders = en2tr_problem.feature_encoders(data_dir)

ckpt_path = tf.train.latest_checkpoint(ckpt_dir)



# Encoder ve Decoder Fonksiyonlarını kuralım


In [20]:
# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id 
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["inputs"].decode(np.squeeze(integers)) 

In [21]:
# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_tpu"

# Other setup
Modes = tf.estimator.ModeKeys

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="translate_en_tr")

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
translate_model = registry.model(model_name)(hparams, Modes.EVAL)

INFO:tensorflow:Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.label_smoothing to 0.0


...

INFO:tensorflow:Setting hparams.symbol_dropout to 0.0


INFO:tensorflow:Setting hparams.attention_dropout to 0.0


INFO:tensorflow:Setting hparams.attention_dropout to 0.0


INFO:tensorflow:Setting hparams.relu_dropout to 0.0


INFO:tensorflow:Setting hparams.relu_dropout to 0.0


In [22]:
# Restore and translate!
def translate(inputs, beam_size = 5, alpha = 0.6, **kwargs):
  encoded_inputs = encode(inputs)
  with tfe.restore_variables_on_create(ckpt_path):
    model_output = translate_model.infer(encoded_inputs, **kwargs)["outputs"]
  if len(model_output.shape) == 2:
    return decode(model_output)
  else:
    return [decode(x) for x in model_output[0]]
  
def translate_and_display(input):
  output = translate(input)
  print('\n  '.join(textwrap.wrap("Input: {}".format(input), 80)))
  print()
  print('\n  '.join(textwrap.wrap("Output: {}".format(output), 80)))

## İnputs yerine çevirmek istediğiniz İngilizce veriyi yazarak çeviri uygulamasını test edebilirsiniz.

In [23]:
inputs = "I want to watch a movie tonight"

translate_and_display(inputs)

INFO:tensorflow:Greedy Decoding


INFO:tensorflow:Greedy Decoding


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


...

Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Input: I want to watch a movie tonight

Output: Bu gece film izlemek istiyorum.


In [30]:
inputs = "I would like to tell you about how to become a successful student.\
 Firstly, you must have a goal."

translate_and_display(inputs)

Input: I would like to tell you about how to become a successful student.
  Firstly, you must have a goal.

Output: Nasıl başarılı bir öğrenci olacağını sana söylemek isterim ama bir aman
  olmalı.


In [31]:
inputs = "The threat comes as press freedom campaigners and charity groups warn \
the government in an open letter that UK courts are being used to “intimidate \
and silence” journalists working in the public interest."

translate_and_display(inputs)

Input: The threat comes as press freedom campaigners and charity groups warn the
  government in an open letter that UK courts are being used to “intimidate and
  silence” journalists working in the public interest.

Output: Söz konusu tehdit, basın özgürlüğü kampanyacıları ve yardım örgütlerinin
  hükümeti, İngiliz mahkemelerinin, kamu yararına çalışan gazetecileri "sindirmeye
  ve susturmaya" alıştığı açık bir mektupta uyarmaları üzerine geldi.


In [32]:
inputs = "Alexandria Ocasio-Cortez called for a “9/11-style commission” to \
investigate child separation on the border with Mexico on Saturday, and said \
the US government has a life-long responsibility to children it severed from \
their parents, to provide them with mental health support."


translate_and_display(inputs)

Input: Alexandria Ocasio-Cortez called for a “9/11-style commission” to
  investigate child separation on the border with Mexico on Saturday, and said the
  US government has a life-long responsibility to children it severed from their
  parents, to provide them with mental health support.

Output: Alexandria Ocasio-Cortez Cumartesi günü Meksika sınırındaki çocuk
  ayrımını araştırması için "9/11 tarzı bir komisyon" çağrısında bulundu ve ABD
  hükümetinin ailelerinden gelen çocuklara olan sağlığına yönelik ömür boyu bir
  sorumluluk taşıdığını söyledi.
